<h1>Segmenting and Clustering Neighborhoods in Toronto</h1>
<h4>By: Alexander Stetzer</h4>

In [3]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import requests 
from bs4 import BeautifulSoup

<h2>Part One:Toronto Neighborhoods and Postal Codes DataFrame</h2>

In [5]:
#url of the postal codes for toronto and processing of url to useable html
url = 'http://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
html = requests.get(url)
d = html.text

#Creation of the BeautifulSoup object
soup = BeautifulSoup(d, 'html5lib')


In [6]:
#Empty list used to gather all of the Postal Data
toronto_list = []
postal_table = soup.find('table')

#Loop to go through the table data and extract the Postal Codes, Boroughs, and Neighborhoods of Toronto
for item in postal_table.findAll('td'):
    code = {}
    if item.span.text == 'Not assigned': #used to remove all of the postal codes with no Borough assignment
        pass
    else:
        code['Postal Code'] = item.p.text[:3]
        code['Borough'] = item.span.text.split('(')[0]
        code['Neighborhood'] = item.span.text.split('(')[1].replace(' /', ',').replace(')','').strip(' ')
        toronto_list.append(code)
    
#Creation of the DataFrame from the list created using the previous loop
toronto_df = pd.DataFrame(toronto_list)
toronto_df['Borough']=toronto_df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

print(toronto_df.head())

  Postal Code           Borough                      Neighborhood
0         M3A        North York                         Parkwoods
1         M4A        North York                  Victoria Village
2         M5A  Downtown Toronto         Regent Park, Harbourfront
3         M6A        North York  Lawrence Manor, Lawrence Heights
4         M7A      Queen's Park     Ontario Provincial Government


In [7]:
print(toronto_df.shape)

(103, 3)


<h2>Part Two: Latitude and Longitude of Neighborhoods</h2>

In [2]:
#!pip install geocoder
#import geocoder